In [30]:
import numpy as np
from functools import cmp_to_key
from PIL import Image
import colorsys
from pathlib import Path
import glob
from IPython.display import clear_output
import ray
import time

### Get frames for a certian range of frames and save it in scene

In [31]:
path = "framesCinematograph/"
startFrame = 70796
endFrame = 77094
nframes = np.array(glob.glob(path+"*"))
scene = np.array([])

imagePIL = Image.open(nframes[0], "r")

for e in nframes:
    if (int(e.split("_")[1].split(".")[0]) >= startFrame and
        int(e.split("_")[1].split(".")[0]) <= endFrame):
        scene = np.append(scene, e)
        
print(len(nframes), "frames found in total")
print(len(scene), "frames found in scene")

288 frames found in total
0 frames found in scene


#### Sort functions From pixelSort.ipynb

In [32]:
# From pixelSort.ipynb
def get_image_pixel(imagePIL, resize="None"):
    if resize == "None":   
        return imagePIL, list(imagePIL.getdata())# np.array(imagePIL.getdata()) #list(imagePIL.getdata())
    else:
        org_size = imagePIL.size
        pixelate_lvl = resize
        imagePIL = imagePIL.resize(size=(org_size[0] // pixelate_lvl, org_size[1] // pixelate_lvl), resample=0)# imagePIL.resize((resize,resize), resample=Image.BILINEAR)
        return imagePIL, np.array(imagePIL.getdata()) # list(imagePIL.getdata())

def sortColors(colors, method):
    colors = (colors / 255) # colorsys uses 0 - 1 range 
    hsv_values = np.apply_along_axis(RGBToHSV, axis=1, arr=colors)

    if method == "HUE":
        hsv_values = hsv_values[hsv_values[:, 0].argsort()]
    elif method == "SATURATION":
        hsv_values = hsv_values[hsv_values[:, 1].argsort()]
    elif method == "BRIGHTNESS":
        hsv_values = hsv_values[hsv_values[:, 2].argsort()]
        
    rgb_values = np.apply_along_axis(HSVToRGB, axis=1, arr=hsv_values)
    rgb_values = (rgb_values * 255).astype(int) # convert color range back to 0 - 255
    return rgb_values
    
def RGBToHSV(color):
    return colorsys.rgb_to_hsv(color[0], color[1], color[2])
   
def HSVToRGB(color):
    return colorsys.hsv_to_rgb(color[0], color[1], color[2])

### sort frames and save to folder

In [9]:
# not using this since it's single core
for i, f in enumerate(nframes):
    image_path = f
    folder = "framesCinematographSorted/"
    filename = f.split("/")[-1]

    localFiles = np.array(glob.glob(folder+"*"))
    found = False
    for item in localFiles:
        if item.split("/")[-1] == filename:
            found = True
            print(i,": found")
            
    if found == False:
        imagePIL = Image.open(image_path, "r")

        im, pixel_values = get_image_pixel(imagePIL, 1)

        sortedImage = sortColors(pixel_values, "HUE")

        im.putdata(list(map(tuple, sortedImage)))
        im.resize(imagePIL.size, resample=0)
        im.save(Path(folder+filename))
        print(i,": sorted")
        
    clear_output(wait=True)

KeyboardInterrupt: 

### Sort frames parallel

In [33]:
ray.shutdown()
ray.init()

start_time = time.time()

@ray.remote
def f(x):
    image_path = nframes[x]
    imagePIL = Image.open(image_path, "r")

    im, pixel_values = get_image_pixel(imagePIL, 1)

    sortedImage = sortColors(pixel_values, "HUE")
    
    return sortedImage

# Start range() tasks in parallel.
result_ids = []
for i in range(len(nframes)):
    result_ids.append(f.remote(i))
    
# Wait for the tasks to complete and retrieve the results.
results = ray.get(result_ids)  # [0, 1, 2, 3]
print("--- took %s seconds ---" % (time.time() - start_time))

(f pid=8876) 
(f pid=8866) 
(f pid=8875) 
(f pid=8867) 
(f pid=8869) 
(f pid=8876) 
--- took 1884.1740047931671 seconds ---


In [34]:
print(len(results))
nr = np.array(results)
np.save('gridMatrixPixels.npy', nr)

288


<ipython-input-34-c065cb26fb9b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  nr = np.array(results)


### save frames to folder

In [38]:
nr = np.array(results)
folder = "framesCinematographSorted/"
im = Image.new("RGB", imagePIL.size, (255, 255, 255))

for i, n in enumerate(nr):
    filename = "m_"+str(i).zfill(4)+".jpg"
    im.putdata(list(map(tuple, n)))
    im.save(Path(folder+filename))

<ipython-input-38-6de2f28cc927>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  nr = np.array(results)
